In [2]:
import pandas as pd
import requests
import json

In [10]:
METADATA_API = "https://maayanlab.cloud/sigcom-lincs/metadata-api/"
DATA_API = "https://maayanlab.cloud/sigcom-lincs/data-api/api/v1/"

input_gene_set = {
#     # CMS2 colotype
#     "up_genes": ["POFUT1", "QPRT", "PLAGL2", "TP53RK", "CEL", "DDX27"],
#     "down_genes": ["LYZ", "TRIB2", "DUSP4", "FSCN1", "IKZF4"]
    
    # CMS2 colotype + GALNT12 + APC
    "up_genes": ["POFUT1", "QPRT", "PLAGL2", "TP53RK", "CEL", "DDX27", 'GALNT12'],
    "down_genes": ["LYZ", "TRIB2", "DUSP4", "FSCN1", "IKZF4", 'APC']
    
#     # CMS2 colotype + APC
#     "up_genes": ["POFUT1", "QPRT", "PLAGL2", "TP53RK", "CEL", "DDX27"],
#     "down_genes": ["LYZ", "TRIB2", "DUSP4", "FSCN1", "IKZF4", 'APC']
}

all_genes = input_gene_set["up_genes"] + input_gene_set["down_genes"]

payload = {
    "filter": {
        "where": {
            "meta.symbol": {
                "inq": all_genes
            }
        },
        "fields": ["id", "meta.symbol"]
    }
}
res = requests.post(METADATA_API + "entities/find", json=payload)
entities = res.json()

for_enrichment = {
    "up_entities": [],
    "down_entities": []
}

for e in entities:
    symbol = e["meta"]["symbol"]
    if symbol in input_gene_set["up_genes"]:
        for_enrichment["up_entities"].append(e["id"])
    elif symbol in input_gene_set["down_genes"]:
        for_enrichment["down_entities"].append(e["id"])

print(json.dumps(for_enrichment, indent=2))

{
  "up_entities": [
    "e5e7afca-258d-48ef-af79-26d749997291",
    "86324105-c420-4b92-9588-2aa2e6b50c19",
    "a5a07ec6-3dfc-464d-af32-5d74fb9c605f",
    "915f4ef6-647d-4bbb-af36-d2dc4f288f5b",
    "8a2763c8-2000-401e-b5ff-26f2b8677b3c",
    "9aec11dc-5988-4f0e-9b26-58de33bebdaf",
    "fe905f3c-6019-490b-a32f-02165a1c3977"
  ],
  "down_entities": [
    "3972b992-34f3-4c13-8744-95b000f0e357",
    "d40f2eac-a8cb-4c2e-8f7a-605e8f4658ef",
    "c2c4164a-3797-4221-a26a-8737250eface",
    "25e69f15-6eb0-4f0e-8bea-9b94355ffea0",
    "ed6bb86b-0691-4776-aee3-24eef7c42219",
    "a8d75668-8ed2-4682-aa3d-2a262515be9f"
  ]
}


In [9]:
payload = {
    "meta": {
        "$validator": "/dcic/signature-commons-schema/v6/meta/user_input/user_input.json",
        **for_enrichment
    },
    "type": "signature"
}
res = requests.post(METADATA_API + "user_input", json=payload)
persistent_id = res.json()["id"]
print("Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/%s"%persistent_id)

Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/4b040547-0142-5bbf-a461-c9aecf2c3b74
